In [52]:
import numpy as np
from scipy.special import erf

#### Справка

$$
\tilde{\Delta} = \sqrt{N} * \max_{x \in \mathbb{R}}{|\tilde {F}(x) - F(x)|}
$$

$$
K(x) = 1 + 2 * \sum_{k=1}^{\infty}(-1)^k \cdot e ^ {-2k^2 x^2}
$$

$$
p-value = P(\Delta \geq \tilde{\Delta} | H_0)  = 1 - K(\tilde{\Delta})
$$

### $\alpha$ = 0,05

## а) Проверить гипотезу о согласии данных с законом равномерного распределения с помощью критерия Колмогорова. Сравнить результаты.

### $H_0$ : равномерное распределение
### $H_1: \overline H_0 $

In [53]:
n = 100
arr = np.array([5, 8, 6, 12, 14, 18, 11, 6, 13, 7])
numbers = np.array([1 for i in range(10)])

emp_cdf = np.array([sum(arr[:i]) for i in range(len(arr)+1)]) / 100
theo_cdf = np.array([sum(numbers[:i]) for i in range(len(numbers))]) / 10

differences = np.maximum(
    np.abs(theo_cdf - emp_cdf[:-1]),
    np.abs(theo_cdf - emp_cdf[1:]) 
)

D = round(np.max(np.sqrt(n) * differences), 5)

print(f"Δ = {D}")

Δ = 1.4


In [54]:
def kolmogorov_distribution(x: float, terms: int = 10000) -> float:
    n = np.arange(1, terms)
    series = 2 * (-1)**n * np.exp(-2 * (n**2) * x**2)
    return 1 + np.sum(series)

p_value = 1 - kolmogorov_distribution(D)
print(f"p-value = {p_value}")

p-value = 0.03968187953811442


p-value < $\alpha$ 

=> отвергаем гипотезу $H_0$

#### Вывод: По критерию $\chi^2$ нет причин отвергать $H_0$, а по критерию Колмогорова мы отвергаем ее, из-за того что есть систематические отклонения от равномерного распределения

## б) Проверить гипотезу о согласии данных с законом нормального распределения с помощью критерия Колмогорова. Сравнить результаты.

### $H_0$ : нормальное распределение
### $H_1: \overline H_0 $

In [56]:
def normal_cdf(x: np.ndarray, mean: float, sigma: float) -> np.ndarray:
    return 0.5 * (1 + erf((x - mean) / (sigma * np.sqrt(2))))

def calculate_bootstrap_deltas(
    true_mean: float,
    true_sigma: float,
    num_iterations: int,
    sample_size: int
) -> np.ndarray:
    deltas = np.empty(num_iterations)
    
    for i in range(num_iterations):
        sample = np.random.normal(true_mean, true_sigma, sample_size)
        sorted_sample = np.sort(sample)
        
        bootstrap_mean = np.mean(sorted_sample)
        bootstrap_sigma = np.std(sorted_sample, ddof=1)
        
        empirical_cdf = np.linspace(0, 1, sample_size + 1)
        
        theory_probs = normal_cdf(sorted_sample, bootstrap_mean, bootstrap_sigma)
        
        deviations = np.maximum(
            np.abs(theory_probs - empirical_cdf[:-1]),
            np.abs(theory_probs - empirical_cdf[1:])
        )
        deltas[i] = np.sqrt(sample_size) * np.max(deviations)
    
    return deltas

NUM_ITERATIONS = 50000
SAMPLE_SIZE = 100
arr = np.array([5, 8, 6, 12, 14, 18, 11, 6, 13, 7])

x_values = np.arange(len(arr))
weights = arr / arr.sum()
mean = np.dot(x_values, weights)
variance = np.dot((x_values - mean)**2, weights) * SAMPLE_SIZE / (SAMPLE_SIZE - 1)
sigma = np.sqrt(variance)

empirical_cdf = np.cumsum(arr) / arr.sum()
empirical_cdf = np.insert(empirical_cdf, 0, 0)

theory_probs = normal_cdf(np.arange(10), mean, sigma)
d0 = np.sqrt(SAMPLE_SIZE) * np.max(
    np.maximum(
        np.abs(theory_probs - empirical_cdf[:-1]),
        np.abs(theory_probs - empirical_cdf[1:])
    )[:10]
)

bootstrap_deltas = calculate_bootstrap_deltas(mean, sigma, NUM_ITERATIONS, SAMPLE_SIZE)
p_value = np.mean(bootstrap_deltas >= d0)

print(f"p-value = {p_value:.5f}")

p-value = 0.01506


#### p-value < $\alpha$ 

=> отвергаем гипотезу $H_0$

#### Вывод: Оба критерия отвергли $H_0$